In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import numpy as np
import pandas as pd 
import torch.optim as optim
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch
from scipy.stats import vonmises
res = 8

In [105]:
class CircleNet(nn.Sequential): 
    def __init__(self, res, dropout, n_out): 
        super(CircleNet, self).__init__()
        self.n_in = res
        self.affine1 = nn.Linear(self.n_in, self.n_in - 6)
        self.drop1 = nn.Dropout(dropout)
        ##self.affine2 = nn.Linear(self.n_in//2, self.n_in//2)
        ##self.drop2 = nn.Dropout(dropout)
        self.affine3 = nn.Linear(self.n_in - 6, n_out)
    def forward(self, x): 
        x = F.relu(self.affine1(x))
        x = self.drop1(x)
        ##x = F.relu(self.affine2(x))
        ##x = self.drop2(x)
        x = F.relu(self.affine3(x))
        x = F.softmax(x, dim = 1)
        return x

In [127]:
## make some uniform circular data
res = 8
num_points = 10000
num_samples = 40
data = np.zeros([num_points,res + 1])
for i in range(num_points): 
    for j in range(num_samples): 
        num = randint(0,res-1)
        data[i][num]+= 1
        data[i][res] = -1
np.savetxt("uniform8ptdata.csv", data, delimiter = ",")

In [ ]:
##reads the data back into a numpy array
data2 = pd.read_csv("uniform.csv")
data2array = data2.values

In [18]:
##This is a class that allows us to load data from csv files and make them into datasets that the dataloader can use
class CircularDataset(Dataset): 
    def __init__(self, data_root): 
        self.data = []
        self.dist_coder = LabelEncoder()
        self.dist_list = []
        for path in data_root: 
            array = pd.read_csv(path)
            
            self.dist_list += [array.values[0,-1]] ##get the distribution type for each file and add it to the list
            if len(self.data) == 0: 
                self.data = array.values
            else:
                print(self.data.shape)
                print(array.values.shape)
                self.data = np.concatenate((self.data,array.values), axis = 0)
                
               
        
        self.dist_coder.fit(self.dist_list)
      
    def __getitem__(self,idx): 
        return self.data[idx][0:-1], self.to_one_hot(self.dist_coder,([self.data[idx,-1]]))[0]
    def __len__(self): 
        return len(self.data)
    
    def to_one_hot(self, codec, values):
        value_idxs = codec.transform(values)
        return torch.eye(len(codec.classes_))[value_idxs]

In [ ]:
[1,2,3][0:-1]

In [ ]:
dataset = CircularDataset(["uniformtest1.csv", "uniformtest2.csv"])

In [ ]:
dataset.__getitem__(0)

In [ ]:
iterator = torch.utils.data.DataLoader(dataset, batch_size = 2, shuffle = True)

In [ ]:
for data, label in iterator: 
    
    print(label)

In [129]:
rate = .005
batchsize = 50
dataset = CircularDataset(["uniform8ptdata.csv", "vonmises8ptdata.csv"])
testdata = CircularDataset(["uniform8pttestdata.csv", "vonmises8pttest.csv"])
iterator = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testiterator = torch.utils.data.DataLoader(testdata, batch_size = batchsize, shuffle = True)
net = CircleNet(8,0,2)
optimizer = optim.Adam(net.parameters(),lr = rate)
running_loss = 0
epochs = 10
net.apply(init_weights)
net.train()
loss_func = nn.CrossEntropyLoss()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    for dist, labels in iterator: 
        
        
        
        optimizer.zero_grad()
        out = net(dist.float()/20)
        
        loss = loss_func(out,torch.max(labels, 1)[1])
       
           
        loss.backward()
        optimizer.step()
        running_loss += loss
    for dist, labels in testiterator: 
        ps = net(dist.float()/20)
        
        i += 1
        top_p, top_class = ps.topk(1, dim=1)
       
        top_p2, top_class2 = labels.topk(1, dim = 1)
        
        equals = top_class == top_class2
        
        accuracy += torch.sum(equals.float())
        
    accuracy = accuracy/i/batchsize
    print(accuracy)
    accuracy = 0
    

(9999, 9)
(9999, 9)
(99, 9)
(99, 9)
0
tensor(0.7250)
tensor(259.1146, grad_fn=<AddBackward0>)
tensor(0.7950)
tensor(223.9254, grad_fn=<AddBackward0>)
tensor(0.8100)
tensor(207.6238, grad_fn=<AddBackward0>)
tensor(0.8050)
tensor(201.7668, grad_fn=<AddBackward0>)
tensor(0.8150)
tensor(198.8269, grad_fn=<AddBackward0>)
tensor(0.8100)
tensor(197.3371, grad_fn=<AddBackward0>)
tensor(0.8200)
tensor(196.4320, grad_fn=<AddBackward0>)
tensor(0.8300)
tensor(195.7659, grad_fn=<AddBackward0>)
tensor(0.8100)
tensor(195.3244, grad_fn=<AddBackward0>)
tensor(0.8200)


In [97]:
for dist, labels in iterator: 
    out = net(dist.float()/100)
    print( str(out) + " True: " + str(labels) + "\n")

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 

...


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.]])

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward>) True: tensor([[0., 1.],
        [1., 0.],
        [0., 1.]])



In [124]:
data = np.zeros([10000,res+1])
for i in range(10000):
    SAMPLE_SIZE = 40
    kappa = 1
    vm_dist = vonmises.rvs(kappa, size=SAMPLE_SIZE)
    vm_dist = (vm_dist + np.pi)/(2*np.pi)*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    
    rotate_amt = randint(0,res - 1)
    data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i,-1] = -3
    

In [125]:
np.savetxt("vonmises8ptdata.csv", data, delimiter = ",")

In [ ]:
print(data[0])

In [8]:
def rotate(vec, num_rotations): 
    vec2 = np.zeros(len(vec))
    for i in range(len(vec)): 
        vec2[(i + num_rotations )% len(vec)] = vec[i]
    return vec2
        

In [45]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.05)